In [1]:
import os
import pandas as pd
import pickle as pkl

from pydeseq2.dds import DeseqDataSet
from pydeseq2.default_inference import DefaultInference
from pydeseq2.ds import DeseqStats
from pydeseq2.utils import load_example_data

In [2]:
counts_df = load_example_data(
    modality="raw_counts",
    dataset="synthetic",
    debug=False,
)

metadata = load_example_data(
    modality="metadata",
    dataset="synthetic",
    debug=False,
)

In [3]:
file_path = "E-MTAB-6863-raw-counts.tsv"
df = pd.read_csv(file_path, sep="\t", comment='#')
df.head()

,Gene ID,Gene Name,ERR2618748,ERR2618749,ERR2618751,ERR2618752,ERR2618753,ERR2618754,ERR2618755,ERR2618756,...,ERR2618798,ERR2618771,ERR2618815,ERR2618778,ERR2618800,ERR2618780,ERR2618799,ERR2618801,ERR2618816,ERR2618750
0,ENSG00000000003,TSPAN6,2230,1191,1592,1936,1506,1717,1489,1882,...,1501,913,722,1346,2215,1593,735,1332,657,829
1,ENSG00000000005,TNMD,12,6,5,2,3,1,1,21,...,0,4,0,5,2,0,0,1,0,2
2,ENSG00000000419,DPM1,404,339,396,326,343,335,292,491,...,354,173,107,420,471,301,126,302,141,165
3,ENSG00000000457,SCYL3,670,422,375,347,417,403,318,528,...,208,194,186,437,555,285,146,444,244,222
4,ENSG00000000460,C1orf112,111,116,97,73,64,108,38,153,...,43,51,46,111,121,65,13,94,61,69


In [10]:
file_path = "E-MTAB-6863-experiment-design.tsv"
edf = pd.read_csv(file_path, sep="\t", comment='#')
edf = edf[edf["Sample Characteristic[disease]"] == "non-alcoholic fatty liver disease"]
edf.head()

,Run,Sample Characteristic[age],Sample Characteristic Ontology Term[age],Sample Characteristic[alcohol consumption],Sample Characteristic Ontology Term[alcohol consumption],Sample Characteristic[alcohol consumption measurement],Sample Characteristic Ontology Term[alcohol consumption measurement],Sample Characteristic[bmi code],Sample Characteristic Ontology Term[bmi code],Sample Characteristic[body mass index],...,Sample Characteristic Ontology Term[smoker],Factor Value[clinical information],Factor Value Ontology Term[clinical information],Factor Value[disease],Factor Value Ontology Term[disease],Factor Value[disease staging],Factor Value Ontology Term[disease staging],Factor Value[infect],Factor Value Ontology Term[infect],Analysed
55,ERR2618803,53 year,NaN,0 grams per week,NaN,0,NaN,1,NaN,37.7,...,NaN,steatosis,NaN,non-alcoholic fatty liver disease,http://www.ebi.ac.uk/efo/EFO_0003095,early,NaN,not applicable,http://purl.obolibrary.org/obo/NCIT_C48660,Yes
56,ERR2618804,51 year,NaN,0 grams per week,NaN,0,NaN,1,NaN,32,...,NaN,steatosis,NaN,non-alcoholic fatty liver disease,http://www.ebi.ac.uk/efo/EFO_0003095,early,NaN,not applicable,http://purl.obolibrary.org/obo/NCIT_C48660,Yes
57,ERR2618805,29 year,NaN,0 grams per week,NaN,0,NaN,1,NaN,30.7,...,NaN,steatosis,NaN,non-alcoholic fatty liver disease,http://www.ebi.ac.uk/efo/EFO_0003095,early,NaN,not applicable,http://purl.obolibrary.org/obo/NCIT_C48660,Yes
58,ERR2618806,37 year,NaN,0 grams per week,NaN,0,NaN,1,NaN,32.7,...,NaN,steatosis,NaN,non-alcoholic fatty liver disease,http://www.ebi.ac.uk/efo/EFO_0003095,early,NaN,not applicable,http://purl.obolibrary.org/obo/NCIT_C48660,Yes
59,ERR2618807,58 year,NaN,0 grams per week,NaN,0,NaN,1,NaN,39.6,...,NaN,steatosis,NaN,non-alcoholic fatty liver disease,http://www.ebi.ac.uk/efo/EFO_0003095,advanced,NaN,not applicable,http://purl.obolibrary.org/obo/NCIT_C48660,Yes


In [22]:
counts_df = df[["Gene ID"] + edf["Run"].tolist()] 
counts_df.shape

(58735, 13)

In [23]:
counts_df.head()

,Gene ID,ERR2618803,ERR2618804,ERR2618805,ERR2618806,ERR2618807,ERR2618808,ERR2618809,ERR2618810,ERR2618811,ERR2618812,ERR2618813,ERR2618814
0,ENSG00000000003,354,1584,1296,2202,1444,2015,1262,1262,1915,2536,1123,705
1,ENSG00000000005,3,8,2,8,3,12,5,4,3,35,5,2
2,ENSG00000000419,87,380,262,486,393,431,287,192,325,439,290,102
3,ENSG00000000457,175,598,379,623,272,677,324,220,351,478,388,102
4,ENSG00000000460,88,239,122,366,134,199,99,37,139,135,109,29


In [24]:
counts_df = counts_df.T
counts_df.columns = counts_df.iloc[0]
counts_df = counts_df.iloc[1:]
counts_df.head()

Gene ID,ENSG00000000003,ENSG00000000005,ENSG00000000419,ENSG00000000457,ENSG00000000460,ENSG00000000938,ENSG00000000971,ENSG00000001036,ENSG00000001084,ENSG00000001167,...,ENSG00000285985,ENSG00000285986,ENSG00000285987,ENSG00000285988,ENSG00000285989,ENSG00000285990,ENSG00000285991,ENSG00000285992,ENSG00000285993,ENSG00000285994
ERR2618803,354,3,87,175,88,45,13910,150,1038,169,...,0,2,0,0,0,0,9,0,9,81
ERR2618804,1584,8,380,598,239,155,59106,495,3467,448,...,0,3,0,0,0,0,19,0,18,92
ERR2618805,1296,2,262,379,122,139,36663,406,2810,399,...,0,1,1,0,0,0,20,0,19,132
ERR2618806,2202,8,486,623,366,150,90130,798,4496,496,...,0,0,0,0,0,0,15,0,23,83
ERR2618807,1444,3,393,272,134,71,46373,349,2224,261,...,0,1,0,0,0,0,4,0,2,47


In [25]:
counts_df.shape

(12, 58735)

In [38]:
metadata = edf[["Run","Sample Characteristic[disease]", "Sample Characteristic[disease staging]"]]
metadata = metadata.rename(columns={"Sample Characteristic[disease]": "condition", 
                         "Sample Characteristic[disease staging]": "stage"})
metadata = metadata.set_index("Run")
metadata

,condition,stage
Run,,
ERR2618803,non-alcoholic fatty liver disease,early
ERR2618804,non-alcoholic fatty liver disease,early
ERR2618805,non-alcoholic fatty liver disease,early
ERR2618806,non-alcoholic fatty liver disease,early
ERR2618807,non-alcoholic fatty liver disease,advanced
ERR2618808,non-alcoholic fatty liver disease,early
ERR2618809,non-alcoholic fatty liver disease,advanced
ERR2618810,non-alcoholic fatty liver disease,early
ERR2618811,non-alcoholic fatty liver disease,advanced


In [39]:
samples_to_keep = ~metadata.condition.isna()
counts_df = counts_df.loc[samples_to_keep]
metadata = metadata.loc[samples_to_keep]

In [40]:
counts_df = counts_df.fillna(0)
genes_to_keep = counts_df.columns[counts_df.sum(axis=0) >= 10]
counts_df = counts_df[genes_to_keep]

In [41]:
counts_df = counts_df.round(0)
counts_df.head()

Gene ID,ENSG00000000003,ENSG00000000005,ENSG00000000419,ENSG00000000457,ENSG00000000460,ENSG00000000938,ENSG00000000971,ENSG00000001036,ENSG00000001084,ENSG00000001167,...,ENSG00000285958,ENSG00000285967,ENSG00000285973,ENSG00000285974,ENSG00000285976,ENSG00000285979,ENSG00000285986,ENSG00000285991,ENSG00000285993,ENSG00000285994
ERR2618803,354,3,87,175,88,45,13910,150,1038,169,...,3,80,18,4,53,83,2,9,9,81
ERR2618804,1584,8,380,598,239,155,59106,495,3467,448,...,1,312,39,6,129,208,3,19,18,92
ERR2618805,1296,2,262,379,122,139,36663,406,2810,399,...,2,179,42,8,135,132,1,20,19,132
ERR2618806,2202,8,486,623,366,150,90130,798,4496,496,...,0,254,28,18,221,197,0,15,23,83
ERR2618807,1444,3,393,272,134,71,46373,349,2224,261,...,0,136,6,4,137,89,1,4,2,47


In [42]:
inference = DefaultInference(n_cpus=8)
dds = DeseqDataSet(
    counts=counts_df,
    metadata=metadata,
    design_factors="stage",
    refit_cooks=True,
    # inference=inference,
)

In [43]:
dds.deseq2()

Fitting size factors...
... done in 0.05 seconds.

Fitting dispersions...
... done in 26.92 seconds.

Fitting dispersion trend curve...
... done in 1.37 seconds.

Fitting MAP dispersions...
... done in 34.11 seconds.

Fitting LFCs...
... done in 16.09 seconds.

Calculating cook's distance...
... done in 0.08 seconds.

Replacing 95 outlier genes.

Fitting dispersions...
... done in 0.11 seconds.

Fitting MAP dispersions...
... done in 0.13 seconds.

Fitting LFCs...
... done in 0.11 seconds.



In [44]:
print(dds)

AnnData object with n_obs × n_vars = 12 × 32820
    obs: 'condition', 'stage'
    uns: 'trend_coeffs', 'disp_function_type', '_squared_logres', 'prior_disp_var'
    obsm: 'design_matrix', 'size_factors', '_mu_LFC', '_hat_diagonals', 'replaceable'
    varm: '_normed_means', 'non_zero', '_MoM_dispersions', 'genewise_dispersions', '_genewise_converged', 'fitted_dispersions', 'MAP_dispersions', '_MAP_converged', 'dispersions', '_outlier_genes', 'LFC', '_LFC_converged', 'replaced', 'refitted'
    layers: 'normed_counts', '_mu_hat', 'cooks', 'replace_cooks'


In [45]:
stat_res = DeseqStats(dds, inference=inference)

In [46]:
stat_res.summary()
stat_res.results_df[:10]

Running Wald tests...
... done in 8.60 seconds.



Log2 fold change & Wald test p-value: stage early vs advanced
                    baseMean  log2FoldChange     lfcSE      stat    pvalue  \
Gene ID                                                                      
ENSG00000000003  1371.307061       -0.280382  0.289466 -0.968617  0.332736   
ENSG00000000005     6.180549        0.816612  0.665761  1.226584  0.219979   
ENSG00000000419   277.390496       -0.521165  0.203386 -2.562450  0.010394   
ENSG00000000457   334.924115       -0.110247  0.150198 -0.734010  0.462943   
ENSG00000000460   121.773641       -0.128984  0.339547 -0.379869  0.704043   
...                      ...             ...       ...       ...       ...   
ENSG00000285979    99.703302       -0.022007  0.407324 -0.054028  0.956913   
ENSG00000285986     1.171463        0.701365  1.246358  0.562732  0.573617   
ENSG00000285991     9.274105        0.005604  0.499972  0.011208  0.991057   
ENSG00000285993     8.980395        0.453787  0.539810  0.840642  0.400549   
EN

,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj
Gene ID,,,,,,
ENSG00000000003,1371.307061,-0.280382,0.289466,-0.968617,0.332736,0.851266
ENSG00000000005,6.180549,0.816612,0.665761,1.226584,0.219979,0.810839
ENSG00000000419,277.390496,-0.521165,0.203386,-2.562450,0.010394,0.422490
ENSG00000000457,334.924115,-0.110247,0.150198,-0.734010,0.462943,0.895326
ENSG00000000460,121.773641,-0.128984,0.339547,-0.379869,0.704043,0.960047
ENSG00000000938,92.257269,0.060252,0.211550,0.284813,0.775787,0.973052
ENSG00000000971,39042.787460,-0.392000,0.308779,-1.269518,0.204256,0.802009
ENSG00000001036,467.770043,-0.029579,0.294869,-0.100313,0.920096,0.990852
ENSG00000001084,2437.319458,-0.065499,0.165100,-0.396720,0.691574,0.956779


In [47]:
results_df = stat_res.results_df.sort_values(by=["padj"])
results_df.head(100)

,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj
Gene ID,,,,,,
ENSG00000169031,114.024484,-1.994185,0.300464,-6.637017,3.200941e-11,9.264163e-07
ENSG00000138448,1101.630789,-0.815118,0.136384,-5.976653,2.277685e-09,3.296038e-05
ENSG00000182836,108.645002,-1.745860,0.298771,-5.843462,5.112691e-09,4.932383e-05
ENSG00000178695,449.062468,-0.820733,0.145040,-5.658661,1.525585e-08,1.103837e-04
ENSG00000104435,23.640789,-3.459058,0.638600,-5.416626,6.073428e-08,2.444410e-04
...,...,...,...,...,...,...
ENSG00000058085,22.375324,-2.297080,0.607619,-3.780458,1.565399e-04,4.744832e-02
ENSG00000164144,313.623140,-0.560234,0.148639,-3.769089,1.638445e-04,4.888647e-02
ENSG00000172667,923.930147,-0.915664,0.243403,-3.761933,1.686052e-04,4.929728e-02


In [54]:
sum(results_df.padj < 0.05)

99

In [74]:
gene_list = df['Gene Name'][(df['Gene ID'].isin(results_df.iloc[:99].index.tolist())) & (~df['Gene Name'].isna())]

In [75]:
"','".join(gene_list.tolist())

"MTMR11','DPEP1','VCAN','EPHA3','LAMC2','BCAT1','GPC4','MOXD1','COL4A4','COL19A1','IGFALS','FGF14','CAPN15','FZD3','NIPAL2','STMN2','TUSC3','CAV2','ASPN','CDH23','SLC38A1','ENPP5','CDH6','CLIP4','EFEMP1','SPP1','PLXDC2','CXCL6','ABCC4','TWSG1','EDA2R','MAP1B','SLC6A11','TTC9','DTNA','PDGFRA','PPP1R1A','LRRC1','MMP7','ITGAV','CCNG2','LUM','UBE2Q2','CYP1A2','IGSF3','ARL6IP5','DCDC2','TTBK1','SLC2A12','SSC4D','ZNF711','HOMER1','CACNA2D1','ANKRD29','ANGPT1','CLSTN2','TPCN2','CAPN2','LRATD1','IFI16','MAP9','TMEM144','ARFIP1','PACSIN3','CYYR1','C18orf54','COL4A3','ROBO1','TMEM154','PYGO1','ID4','ZMAT3','MUC13','B3GNT5','KCTD12','TH','TLCD5','ZSCAN22','PLCXD1','PLCXD3','MEX3B','GABRA5','PCLO','SAXO2','LAMA2','SULT1C2','IGLV3-21','OR2I1P','AADACP1','MYH4','MARCKS','PRRT1B"